In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
#Getting the fundamentals data
df = pd.read_csv("SF0_20170108.csv",header=None)
df.columns = ['label','date','value']

#wee bit of cleanup on the labels
dfTemp = pd.DataFrame(df['label'].str.split('_').tolist(),columns=['ticker','attributes','MRY'])
df['ticker'],df['attributes'] = dfTemp['ticker'],dfTemp['attributes']
df = df.drop("label",axis=1)
df

,date,value,ticker,attributes
0,2013-12-31,0.000000e+00,AAC,ACCOCI
1,2014-12-31,0.000000e+00,AAC,ACCOCI
2,2015-12-31,0.000000e+00,AAC,ACCOCI
3,2013-12-31,2.952900e+07,AAC,ASSETSC
4,2014-12-31,7.992200e+07,AAC,ASSETSC
5,2015-12-31,8.652400e+07,AAC,ASSETSC
6,2013-12-31,5.210900e+07,AAC,ASSETSNC
7,2014-12-31,6.603000e+07,AAC,ASSETSNC
8,2015-12-31,2.295250e+08,AAC,ASSETSNC
9,2013-12-31,8.163800e+07,AAC,ASSETS


In [3]:
#extracting the months and years out of dates and dropping the latter
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df = df.drop('date',axis=1)

#loading up the historical price data
prices = pd.read_csv('WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv',usecols=['ticker','date','adj_close'])
prices['date'] = pd.to_datetime(prices['date'])
prices['year'] = prices['date'].dt.year
prices['month'] = prices['date'].dt.month

In [7]:
#averaging the prices of the fiscal year-end month
avgP = prices.groupby(['ticker','year','month'],as_index=False).mean()

#Calculating returns per ticker
prices = prices.set_index('date')
prices_shrt = prices[prices['year']>2011]
prices_shrt['value'] = 1.0
prices_shrt['value'] = prices_shrt.groupby(['ticker', 'year','month']).pct_change()
prices_shrt['value'] = prices_shrt['value'] + 1
monthlyR = prices_shrt.groupby(by=['ticker','year','month'],as_index=False)['value'].prod()
monthlyR['value'] = monthlyR['value'] -1
monthlyR['attributes'] = 'monthlyReturn'
df = df.append(monthlyR)


#average year-end prices
fuit = df.groupby(['ticker','year','month'],as_index=False).count()
fuit['attributes'] = 'price'
fuit = pd.merge(fuit,avgP, on = ['ticker','year','month'], how='left')
fuit = fuit.drop(['value'],axis=1)
fuit.columns = ['ticker','year','month','attributes','value']

#appending the calculated data to the master dataframe
df = df.append(fuit)

#drop the NaN
df = df[np.isfinite(df['value'])]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [11]:
#objective
#Predict 5 year returns with fundamentals
## Find the factors that contribute to those return
## Which ex ante valuation metrics are good proxies to predict 5y growth?
#### TO DO
#### Calculate 5y growth rate per ticker
#### Calculate ex ante metrics (beginning of observation period)

## Which periodic metric is the most strongly linked to the ending YoY growth?
#### TO DO
#### Calculate 5y growth rate per ticker
#### Calculate average metrics for the period



#INDUSTRY EXTRACTOR
#https://mktstk.com/2015/03/03/sic-lookup-by-stock-symbol/

#df.to_csv("fuit.csv")
pivoted['DEBT']

ticker  year
A       2009    2.905000e+09
        2010    3.691000e+09
        2011    2.185000e+09
        2012    2.362000e+09
        2013    2.699000e+09
        2014    1.663000e+09
        2015    1.655000e+09
        2016    1.912000e+09
AAC     2012             NaN
        2013    4.307500e+07
        2014    2.864100e+07
        2015    1.451410e+08
AAL     2009    1.169700e+10
        2010    1.113600e+10
        2011    8.220000e+09
        2012    8.535000e+09
        2013    1.679900e+10
        2014    1.772000e+10
        2015    2.056100e+10
AAN     2009    5.504400e+07
        2010    4.179000e+07
        2011    1.537890e+08
        2012    1.415280e+08
        2013    1.427040e+08
        2014    6.060820e+08
        2015    6.067460e+08
AAP     2010    2.042710e+08
        2011    3.589040e+08
        2012    6.050880e+08
        2013    1.053584e+09
                    ...     
ZION    2013    2.613923e+09
        2014    1.336505e+09
        2015    1.159353e+09
Z

In [15]:
df['attributes'].unique()

array(['ACCOCI', 'ASSETSC', 'ASSETSNC', 'ASSETS', 'BVPS', 'CAPEX',
       'CASHNEQUSD', 'CASHNEQ', 'COR', 'CURRENTRATIO', 'DEBTUSD', 'DEBT',
       'DEPAMOR', 'DE', 'DILUTIONRATIO', 'DPS', 'EBITDAUSD', 'EBITDA',
       'EBITUSD', 'EBIT', 'EBT', 'EPSDIL', 'EPSUSD', 'EPS', 'EQUITYUSD',
       'EQUITY', 'FCFPS', 'FCF', 'GP', 'INTANGIBLES', 'INTEXP', 'INVCAP',
       'INVENTORY', 'LIABILITIESC', 'LIABILITIESNC', 'LIABILITIES',
       'NCFCOMMON', 'NCFDEBT', 'NCFDIV', 'NCFF', 'NCFI', 'NCFO', 'NCFX',
       'NCF', 'NETINCCMNUSD', 'NETINCCMN', 'NETINCDIS', 'NETINC',
       'PAYABLES', 'PB', 'PREFDIVIS', 'RECEIVABLES', 'RETEARN',
       'REVENUEUSD', 'REVENUE', 'RND', 'SGNA', 'SHARESWADIL', 'SHARESWA',
       'TANGIBLES', 'TAXEXP', 'TBVPS', 'WORKINGCAPITAL', 'monthlyReturn',
       'price'], dtype=object)

In [6]:
results = pd.DataFrame()
pivoted = pd.pivot_table(df, values='value', index=['ticker', 'year'],columns='attributes')

#relevant ratios

#financial performance
results['ROE'] = pivoted['NETINC']/pivoted['EQUITY']
results['ROA'] = pivoted['EBITDA']*(1-pivoted['TAXEXP']/pivoted['EBT'])/pivoted['ASSETS']
results['ROIC'] = (pivoted['EBIT']*(1-pivoted['TAXEXP']/pivoted['EBT']))/(pivoted['EQUITY']+pivoted['DEBT']-pivoted['CASHNEQ'])
results['FCFEtoEq'] = (pivoted['CF']-pivoted['CAPEX'])/pivoted['EQUITY']
results['FCFFtoOpAss'] = (pivoted['CF']+pivoted['INTEXP']*(1-pivoted['TAXEXP']/pivoted['EBT'])-pivoted['CAPEX'])/pivoted['EQUITY']

#credibility
results['NOA'] = pivoted['REVENUE']/pivoted['ASSETS']
results['CashAcc'] = pivoted['REVENUE']/pivoted['ASSETS']
results['AssAcc'] = pivoted['REVENUE']/pivoted['ASSETS']

#Valuation
results['PEdil'] = pivoted['price']/(pivoted['NETINC']/pivoted['SHARESWADIL'])
results['PtoCF'] = pivoted['price']/(pivoted['CF']/pivoted['SHARESWADIL'])
results['PtoBV'] = pivoted['price']/pivoted['BVPS']
results['PtoFCF'] = pivoted['price']/pivoted['FCFPS']
results['mktCap'] = pivoted['price']*pivoted['SHARESWA']


#debt
results['DE'] = pivoted['DEBT']/pivoted['EQUITY']

#operational efficiency
results['CashConvCycle'] = pivoted['REVENUE']/pivoted['ASSETS']
results['AssTurnover'] = pivoted['REVENUE']/pivoted['ASSETS']

#growth
results['NIg'] = pivoted['REVENUE']/pivoted['ASSETS']
results['BVg'] = pivoted['REVENUE']/pivoted['ASSETS']
results['EBITDAg'] = pivoted['REVENUE']/pivoted['ASSETS']
results['CFg'] = pivoted['REVENUE']/pivoted['ASSETS']

results.to_csv('results.csv')

,ticker,year
0,A,2009
1,A,2010
2,A,2011
3,A,2012
4,A,2013
5,A,2014
6,A,2015
7,A,2016
8,A,2017
9,AA,2016


In [16]:
#metrix['ROE'] = df['value'].loc[(df['attributes']=='NETINCCMN') & (df['year']==2015)]
metrix.shape

(24974, 2)